# Jupyter Notebook for the Data Analysis of Alkyne Semihydrogenation

This notebook is inspired/based on work and analyses done in:

* S. H. Newman-Stonebraker, A. G. Doyle *et al.*, *Science* **2021**, *374*, 301-308. Jupyter notebook available under: https://github.com/SigmanGroup/Threshold  
* J. Dotson, M. Sigman *et al.*, *J. Am. Chem. Soc.* **2023**, *145, 1*, 110-121. Jupyter notebook available under: https://github.com/SigmanGroup/Multiobjective_Optimization

# 1. Single-Node Decision Tree (Threshold Analysis)

## Reading data
### Choosing input files

- Sets up script to import data and parameters from excel
- populate this cell based on the excel spreadsheet that you want to be read
- populate y_cut with threshold value

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn import metrics
from matplotlib.colors import ListedColormap

In [ ]:
comp_file = 'DataSet_AllAlkynes_TMS' # parameter excel file name 
comp_sheet = "DataSheet" # parameter excel sheet name
num_par = 44 # number of parameters
par_start_col = 3  # 0-indexed, first column number with descriptor values
comp_num_samples = 2716 # Number of substrates in sheet
y_label_col_comp = 0 # 0-indexed, the number of the column with the substrate ids

exp_file = 'DataSet_Tested' # data excel file name
exp_sheet = 'DataSheet' # data excel sheet name
exp_num_samples = 31 # include all the ligands, the empty rows will be removed later
response_col = 2 # 0-indexed, the column containing the experimental results
y_label_col_exp= 1 # 0-indexed, the number of the column with the substrate ids

y_cut = 0.033   # this sets the threshold value for 1/0 

### Sorting data

- Imports parameters and data and converts continuous data to binary data
- No user input required

In [ ]:
#********************************************************************
#Creation of dataframe, extraction of descriptors and output variables
#********************************************************************

# make pd df for the descriptors
compinp = pd.read_excel(comp_file+".xlsx",comp_sheet,header=0,index_col=y_label_col_comp,nrows=comp_num_samples+1,usecols=list(range(0,(num_par+par_start_col))),engine='openpyxl')
compinp = compinp.drop(['SMILES'],axis=1)

compinp.dropna(inplace=True)
par_start_col = 1
compinp.index = compinp.index.map(str)

# make pd df for the experimental file
expinp = pd.read_excel(exp_file+".xlsx",exp_sheet,header=2,index_col=y_label_col_exp,nrows=exp_num_samples,usecols=list(range(0,response_col+1)),engine='openpyxl')

# names of the descriptors
X_names = list(compinp.iloc[0,par_start_col-1:num_par+par_start_col-1])

# labels for descriptors e.g. x1, x2, x3... make list then take only selection for descriptors
X_labels = list(compinp.columns)[par_start_col-1:num_par+par_start_col-1]

# removes names of the descriptors from compinp
compinp.drop(index=compinp.index[0],inplace=True)

# X_all = np array of descriptor values for all substrates
X_all = np.asarray(compinp[X_labels],dtype=np.float)

# np array of the alkyne ids from the descriptor file
y_labels_comp = np.asarray(list(compinp.index),dtype=str)

# compnan = array of True/False designating if a substrate has a missing descriptor value(s) or not
# nan = not a number. isnan returns True if Nan, in this case for any value in a row (axis of 1 = row).
compnan = np.isnan(X_all).any(axis=1)

# compares the arrays, and keeps the sample in y_labels_comp/X_all if the corresponding value in ~compnan = True.
# ~ means it inverts True and False in compnan. This is removing any substrate missing descriptors.
y_labels_comp,X_all = y_labels_comp[~compnan],X_all[~compnan]

# combines the labels and names of descriptors as a single value in a list e.g. "x1 E_HOMO" 
X_labelname = [" ".join(i) for i in zip(X_labels,X_names)]

# makes a dictionary with key of descriptor label, value of descriptor name
X_labelname_dict = dict(zip(X_labels,X_names))

# heading ('label') for the response column
resp_label = list(expinp.columns)[response_col-1]

# array of the experimental results
y = np.asarray(expinp.iloc[:,response_col-1],dtype=np.float)

# array of all the ligand ids in the experimental file (curated below to give ligands with results only)
y_labels_exp = np.asarray(list(expinp.index),dtype=str)

# array with True for experimental results present, False for none
mask_y = ~np.isnan(y)

# check if each value of y_labels_exp (ligand ids in exp file) is also in y_labels_comp (ligand ids in descriptor file),
# to give an array with True/False if a match is found (i.e. do we have the descriptors we need) 
mask_X = np.array([True if i in y_labels_comp else False for i in y_labels_exp])

# compares the two arrays, if same value is True in both then value = True in mask, otherwise = False.
# i.e. does the ligand have an experimental result and descriptors
mask = mask_y&mask_X

# ligands_removed is a list of ligands that had zero-values
count = 0
ligands_removed = []
for i in list(mask):
    if not i:
        ligands_removed.append(y_labels_exp[count])
    count += 1

print("Number of entries in experimental file before removing empty cells: {}".format(len(y)))
print("Removing {} entries with empty cells".format(len(y)-sum(mask)))
print('Entries removed: ', ligands_removed)

# remove all nan values from y, leaving experimental results
y = y[np.array(mask)]

# Convert reaction data to binary 1/0 based on y_cut
y_class = np.array([1 if result > y_cut else 0 for result in y])

# cut y_labels to only have ids for alkynes with results
y_labels = y_labels_exp[np.array(mask)]

# X = array of descriptor values for the alkynes with experimental results
X = np.asarray(compinp.loc[y_labels],dtype=np.float)



verbose = True
if verbose:
    print("Shape of descriptors file for all alkynes: {}".format(X_all.shape))
    print("Last three ids in the descriptor file: {}".format(y_labels_comp[-3:]))
    print("Shape of descriptors file for alkynes with experimental results: {}".format(X.shape))
    print("Shape of results file results (only alkynes with experimental results): {}".format(y.shape)) 
    print("Shape of results file labels (only alkynes with experimental results): {}".format(y_labels.shape))
    print("First descriptor cell (for alkynes with experimental results): {}".format(X[0,0]))
    print("Last descriptor cell (for alkynes with experimental results):  {}".format(X[-1,-1]))
    print('Alkynes with results:',y_labels)
    print(len(y),' Experimental results:', " : ", y)

In [ ]:
#********************************************************************
#Single Node Decision Tree Analysis for all Features
#********************************************************************

#class_weight = {0:1,1:20}   # define class weights
class_weight = "balanced"
accuracy_cutoff = 0.9 #define accuracy cutoff to only get selected features

target_names = ['0 ("negatives")',  '1 ("positives")']

#set which parameters/features to iterate through
features = range(0,len(X_labels))

#threshold analysis for selected features
for f_ind in features:
    feature = X_labels[f_ind] 
    
    dt = DecisionTreeClassifier(max_depth=1,class_weight=class_weight).fit(X[:,f_ind].reshape(-1, 1), y_class)  
    if dt.score(X[:,f_ind].reshape(-1, 1), y_class) >= accuracy_cutoff:
        print("N = {}\nFeature: {}\nDecision threshold = {:.2f}\nAccuracy: {:.2f}\nf1_score: {:.2f}\nMCC: {:.2f} \nRecall: = {:.2f}\nClassification Report: \n{}".format(len(y),feature,
        dt.tree_.threshold[0],
        dt.score(X[:,f_ind].reshape(-1, 1), y_class),
        metrics.f1_score(y_class,dt.predict(X[:,f_ind].reshape(-1, 1))),
        metrics.matthews_corrcoef(y_class,dt.predict(X[:,f_ind].reshape(-1, 1))),
        metrics.recall_score(y_class, dt.predict(X[:,f_ind].reshape(-1, 1)), pos_label=1, average='binary'),
        metrics.classification_report(y_class, dt.predict(X[:,f_ind].reshape(-1, 1)), target_names=target_names),              
        ))
    
        # begin plot
        dt_plt = DecisionTreeClassifier(max_depth=1,class_weight=class_weight).fit(X[:,f_ind].reshape(-1, 1), y_class)
        n_classes = 2
        plot_step = 0.02
    
        # define plot axes limits here as appropriate 
        x_min, x_max = X[:,f_ind].min(), X[:,f_ind].max()
        y_min, y_max = y.min(), y.max()
    
        # set plot colors
        cMap_background = ListedColormap(['white', '#ccebc5',]) # color for the backgrounds: light green and red
        cMap_points = ListedColormap(["r","g"]) # the color for each class of the actual data points (i.e. inactive/active)). "rg" is red,green
    
        # plot code
        fig = plt.figure(figsize=(6, 6)) 
        ax = fig.add_subplot()
        for axis in ['top','bottom','left','right']:
            ax.spines[axis].set_linewidth(3)
        ax.tick_params(length=12,width=3)
        ax.set_ylim([y_min-0.05*y_max,y_max*1.1])
        ax.set_xlim([x_min-0.05*x_max,x_max*1.1])

        plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=3.5)
        plt.xlabel(feature,fontsize=18)
        plt.ylabel(r"mmol $H_{2}$ / h",fontsize=18)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)

        ax.axhspan(ymin=y_min-0.05*y_max,ymax=y_cut,color='grey',alpha=0.15)
        ax.axvspan(xmin=x_min-0.05*x_max,xmax=dt.tree_.threshold[0],color="grey",alpha=0.15)
    
        plt.scatter(X[:,f_ind],y,c=y_class,cmap=cMap_points,edgecolor="black",s=75)
    
        # Print plot
        plt.show()
    
        print('-------------------------------------------------------')

# 2. Logistic Regression

In [ ]:
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split,RepeatedKFold,cross_val_score,cross_validate
from logreg_stats import calc_mcfad, calc_mcfadden_R2, precision_recall_f1_score, test_accuracy_score, kfold_logreg 
import multiprocessing
nproc = max([1,multiprocessing.cpu_count()-2])

randomstate = 42
import Logistic_Regression as fsc

## 2.1. Define Plotting Functions
### Define plotting function for one-parameter logistic regression

- No user input required

In [ ]:
def plot_fit_1D(feat, df_combined, save_fig=False):  
    X_train = np.array(df_combined.loc[:, [feat]])
    y_train = np.array(df_combined.iloc[:, -1])
    lr = LogisticRegression().fit(X_train,y_train)
    m, b = lr.coef_[0][0], lr.intercept_[0]
    feat_name = X_labelname_dict[feat]

    x_min, x_max = min(X_train), max(X_train)
    x_range = x_max - x_min
    plot_min, plot_max = float(x_min-0.05*x_range), float(x_max+0.05*x_range)

    fig= plt.figure(figsize=(6,6))
    ax = fig.add_subplot()
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(3)
    ax.tick_params(length=12,width=3)

    ax.tick_params(which='minor', length=6, width=3)

    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=3.5)

    plt.xticks(fontsize=15) 
    plt.yticks(fontsize=15)
    plt.xlabel(feat_name,fontsize=18)
    plt.ylabel('Probability',fontsize=18)
    #plt.locator_params(axis='y', nbins=4)
    #plt.locator_params(axis='x', nbins=5)
    plt.axvline(x=(-b/m),alpha=1,c='black')
    plt.axvline(x=((np.log(3)-b)/m),alpha=1,c='black', linestyle = '--')
    plt.axvline(x=((np.log(1/3)-b)/m),alpha=1,c='black', linestyle = '--')
    plt.scatter(X_train, y_train, label="training", alpha=1,marker='o', c ='Blue', s=150  ,edgecolor='black')
    plt.xlim(plot_min, plot_max)
    plt.ylim(-.05, 1.05)

    x = np.linspace(x_min-0.4*x_range, x_max+0.4*x_range)
    f_x = np.exp(b + m*x)
    y_sigmoid = f_x/(f_x + 1)
    plt.plot(x, y_sigmoid, color = 'black')
    if save_fig:
        fig.savefig(feat + '.tif', dpi=600)
    plt.show()

## 2.2. Univariate correlations and feature/feature plotting

### Univariate Logistic Regression

- Runs univariate logistic regression for all parameters and creates a dataframe of models in decending order of McFadden R2
- No user input required

In [ ]:
results_1_param = pd.DataFrame(columns=['Model', 'Accuracy', 'McFadden_R2', 'Param_name', 'Threshold_Value'])

count = 0
for i in range(len(X_labels)):
    term = X_labels[i]
    X_sel = X[:, i].reshape(-1,1)
    lr = LogisticRegression().fit(X_sel,y_class)
    acc = round(lr.score(X_sel,y_class), 2)
    mcfad_r2 = round(calc_mcfad(X_sel, y_class), 2)
    m, b = lr.coef_[0][0], lr.intercept_[0]
    row_i = {'Model': term, 'Accuracy': acc, 'McFadden_R2': mcfad_r2, 'Param_name': X_labelname_dict[term], 'Threshold_Value': -b/m}
    results_1_param = results_1_param.append(row_i, ignore_index=True)

results_1_param = results_1_param.sort_values('McFadden_R2', ascending=False)
results_1_param.head(10)

### Plot top univariates

- prints top n models (n = num_plots)
- populate num_plots and skipfeatures

In [ ]:
num_plots = 3  #Specify how many univariates to plot
skipfeatures = []  # add any features that you don't want to plot, e.g. ['x1', 'x28']


df_combined = pd.DataFrame(np.hstack((X,y_class[:,None]))) 
newcols = ["x"+str(i+1) for i in df_combined.columns.values]
df_combined.columns = newcols
response = newcols[-1]
df_combined.rename(columns={response:"y"},inplace=True)
df_combined.drop(skipfeatures,axis=1,inplace=True)

for i in range(num_plots):
    param_i = results_1_param.iloc[i].Model
    param_name = X_labelname_dict[param_i]
    threshold = results_1_param.iloc[i].Threshold_Value
    print("{} {}. Threshold value {:.2f}".format(param_i, param_name, threshold))
    print("Accuracy: {:.0f}%".format(100*results_1_param.iloc[i].Accuracy))
    print("McFadden R2: {}".format(results_1_param.iloc[i].McFadden_R2))
    plot_fit_1D(param_i, df_combined, save_fig=False)
    print("\n\n")

### Plot a univariate logistic regression with a user-defined parameter 

- Runs univariate logistic regression for user-defined parameter list
- Populate plot_params

In [ ]:
plot_params = ['x28','x41'] #populate with a list of parameters (e.g. plot_params = ['x1', 'x20', 'x21'])

for param_i in plot_params:
    row = results_1_param[results_1_param.Model == param_i]
    param_name = X_labelname_dict[param_i]
    threshold = float(row.Threshold_Value)
    acc = float(row.Accuracy)
    mcfad_r2 = float(row.McFadden_R2)
    print("{} {}. Threshold value {:.2f}".format(param_i, param_name, threshold))
    print("Accuracy: {:.0f}%".format(100*acc))
    print("McFadden R2: {}".format(mcfad_r2))
    plot_fit_1D(param_i,df_combined)
    print("\n\n")

### Feature vs. another feature

- plots one feature vs another to determine possible correlations
- Select two features to visualize (f_ind_1, f_ind_2)

In [ ]:
f_ind_1 = "x28" # feature of interest 1 (e.g. 'x1')
f_ind_2 = "x9" # feature of interest 2 (e.g. 'x20')

if type(f_ind_1) == str:
    [f_ind_1,f_ind_2] = [X_labels.index(i) for i in [f_ind_1,f_ind_2]]

print(X_labels[f_ind_1], X_names[f_ind_1])
print(X_labels[f_ind_2], X_names[f_ind_2])
print("\n{} samples".format(np.shape(X[:,f_ind_1])[0]))
slope, intercept, r_value, p_value, std_err = stats.linregress(X[:,f_ind_1],X[:,f_ind_2])
fit_line = intercept+slope*X[:,f_ind_1]
print("R^2 = {:.2f}; p-value = {:.2E}".format(r_value**2,p_value))

plt.figure(figsize=(13, 4))

plt.subplot(1,3,1)
plt.hist(X[:,f_ind_1], bins="auto",color="black")
plt.ylabel("frequency")
plt.xlabel(X_names[f_ind_1])
plt.subplot(1,3,2)
plt.hist(X[:,f_ind_2], bins="auto",color="black")
plt.ylabel("frequency")
plt.xlabel(X_names[f_ind_2])

plt.subplot(1,3,3)
plt.scatter(X[:,f_ind_1], X[:,f_ind_2],color="black",marker=".",alpha=0.5,s=100)    
plt.plot(X[:,f_ind_1],fit_line)
plt.xlabel(X_names[f_ind_1])
plt.ylabel(X_names[f_ind_2])
plt.tight_layout()
plt.show()  


# 3. Multivariate Logisitc Regression

### Define plotting function for 2D logistic regression

- No user input required
- colormap_scheme, test_points, train_points, point_color, and point_size can be changed to adjust the appearance of the plot

In [ ]:
colormap_scheme = 'RdBu'
test_points, train_points = 'X', 'o' # marker shape of test set ('*', <, >, x, X, o, O, b, d,. )
point_color = 'Greys'# 'Greens', 'Oranges', 'Reds','YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu', 'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
point_size = 100

def heatmap_logreg(feat1, feat2, df_train, df_test, model, annotate_test = False, annotate_train = False):  
    #Train model on features
    p1_vals = df_train.iloc[:, feat1]
    p2_vals = df_train.iloc[:, feat2]
    p1_vals_test = df_test.iloc[:, feat1]
    p2_vals_test = df_test.iloc[:, feat2]
    lr = LogisticRegression().fit(df_train.iloc[:,[feat1, feat2]],y_train)
    intercept = lr.intercept_[0]
    c1, c2 = lr.coef_[0][0], lr.coef_[0][1]
    p1, p2 = X_names[feat1], X_names[feat2]
    
    if p1=="δ_Cavg":
        p1= r"$\delta_{\mathregular{C}}^{\mathregular{{avg}}}$"
    if p2=="δ_Cavg":
        p2= r"$\delta_{\mathregular{C}}^{\mathregular{{avg}}}$"
    if p1=="NBO_BDE -2":
        p1= r"NBO_BD$_{\mathregular{E-2}}$"
    if p2=="NBO_BDE -2":
        p2= r"NBO_BD$_{\mathregular{E-2}}$"
        
    #Get max/min values for X and Y axes
    max_x, min_x = max(list(p1_vals) + list(p1_vals_test)), min(list(p1_vals) + list(p1_vals_test))
    max_y, min_y = max(list(p2_vals) + list(p2_vals_test)), min(list(p2_vals) + list(p2_vals_test))
    range_x, range_y = abs(max_x - min_x), abs(max_y - min_y)
    max_x_plt, min_x_plt = max_x + 0.1*range_x, min_x - 0.1*range_x
    max_y_plt, min_y_plt = max_y + 0.1*range_y, min_y - 0.1*range_y
    
    #heatmap code
    xx = np.linspace(min_x_plt, max_x_plt, 500)
    yy = np.linspace(min_y_plt, max_y_plt, 500)
    xx,yy = np.meshgrid(xx, yy)
    Xfull = np.c_[xx.ravel(), yy.ravel()]
   
    # Predict probabilities of full grid
    probas = lr.predict_proba(Xfull)
    

    fig= plt.figure(figsize=(7.25,6))
    ax = fig.add_subplot()
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(3)
    ax.tick_params(length=12,width=3)

    ax.tick_params(which='minor', length=6, width=3)

    plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=3.5)
    plt.xticks(fontsize=15) 
    plt.yticks(fontsize=15)
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax.yaxis.set_major_locator(plt.MaxNLocator(4))
    plt.xlabel(p1,fontsize=18)
    plt.ylabel(p2,fontsize=18)

    
    x = np.linspace(min_x - 0.1*range_x, max_x + 0.1*range_x)
    y = -(intercept/c2) - (c1/c2)*x
    y_75 = ((np.log(3)-intercept)/c2) - (c1/c2)*x    #
    y_25 = ((np.log(1/3)-intercept)/c2) - (c1/c2)*x
    
    plt.xlim([min_x - 0.1*range_x, max_x + 0.1*range_x])
    plt.ylim([min_y - 0.1*range_y, max_y + 0.1*range_y])

    plt.plot(x, y, color = 'black')
    plt.plot(x,y_75, color = 'black', linestyle = '--')
    plt.plot(x,y_25, color = 'black', linestyle = '--')
    
    heatmap = plt.imshow(
            probas[:,1].reshape((500, 500)), cmap=colormap_scheme, alpha = 0.5, extent=[min_x_plt, max_x_plt, min_y_plt, max_y_plt],interpolation='nearest', origin="lower"
        ,aspect='auto')
    cbar = plt.colorbar(heatmap)
    cbar.ax.tick_params(labelsize=15)
    cbar.set_label('Probability', size=18)
    plt.clim(0,1)
    plt.locator_params(axis='y', nbins=5)
    plt.locator_params(axis='x', nbins=5)
    plt.scatter(p1_vals, p2_vals, label="training", alpha=1,marker='o', c = df_train.iloc[:, -1], cmap=point_color + '_r', s=200  ,edgecolor='black')
    plt.scatter(p1_vals_test, p2_vals_test, label="test", alpha=1,marker='X' , c = df_test.iloc[:, -1], cmap=point_color + '_r', s=200  ,edgecolor='black')
    #plt.colorbar()
    #cbar.set_label('yield %',rotation=90,size=22,labelpad=20)

    
    if annotate_test == True:
        for i, txt in enumerate(VS):
            label_i = y_labels[txt]
            plt.annotate(label_i, (p1_vals_test[i], p2_vals_test[i]), fontsize='14', c = 'white')
    if annotate_train == True:
        for i, txt in enumerate(TS):
            label_i = y_labels[txt]
            plt.annotate(label_i, (p1_vals[i], p2_vals[i]), fontsize='14', c = 'white')
    plt.tight_layout()
    fig.savefig('Logistic_Regression.tif', dpi=600)
    plt.show()

## 3.1. Train-Test Spit Approaches

 - populate split and test_ratio: For large data set (n>100)

In [ ]:
# Methods to split the data:
# 'random': randomly assigns train/test split
# 'define': give a list of ligand IDs for the training set (TS) and validation set (VS) in the corresponding code section.
# 'none': all samples in TS.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
split = "none"  # use one of the methods outlined above
test_ratio = 0.3 # float from 0.0 to 1.0. Only relevant when split = 'random'

X_sel,y_sel,labels_sel,exclude = X,y_class,y_labels,[]

if split == "random":
    X_train, X_test, y_train, y_test = train_test_split(
        X_sel, y_sel, random_state=randomstate+3, test_size=test_ratio)    
    TS = [np.argwhere(np.all(X==i,axis=1))[0,0] for i in X_train]
    VS = [np.argwhere(np.all(X==i,axis=1))[0,0] for i in X_test]
    
elif split == "define":
    VS = [16,27,25,5,13,9,30,7,10,19]
    TS = [24,6,22,23,18,17,2,12,20,26,8,14,1,15,4,29,21,0,28,3,11]
    
    X_train, y_train,X_test, y_test = X_sel[TS],y_sel[TS],X_sel[VS],y_sel[VS]
    X_train = scaler.fit_transform(X_train)  # fit on training data
    X_test = scaler.transform(X_test)        # apply same transformation to test data

elif split == "none":
    TS, VS = [i for i in range(X_sel.shape[0]) if i not in exclude],[]
    X_train, y_train,X_test, y_test = X_sel[TS],y_sel[TS],X_sel[VS],y_sel[VS]
    X_train = scaler.fit_transform(X_train)  # fit on training data
else: 
    raise ValueError("split option not recognized")

print("TS: {}".format(TS))
print("VS: {}".format(VS))
print("y_mean TS: {:.3f}".format(np.mean(y_train)))
print("y_mean VS: {:.3f}".format(np.mean(y_test)))
print("Shape X_train: {}".format(X_train.shape))
print("Shape X_test:  {}".format(X_test.shape))   
plt.figure(figsize=(6, 6))
hist,bins = np.histogram(y_sel,bins="auto")#"auto"
plt.hist(y_train, bins, alpha=0.5, label='y_train',color="black")
plt.hist(y_test, bins, alpha=0.5, label='y_test')
plt.legend(loc='best',fontsize=14)
plt.xlabel("yield",fontsize=18)
plt.ylabel("N samples",fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.show()

### Prep data

- Prepares data for modelling (descriptors and results)
- populate skipfeatures with parameters that you don't want the model to evaluate

In [ ]:
skipfeatures = []       
df_train = pd.DataFrame(np.hstack((X_train,y_train[:,None])))     
newcols = ["x"+str(i+1) for i in df_train.columns.values]
df_train.columns = newcols
response = newcols[-1]
df_train.rename(columns={response:"y"},inplace=True)
df_train.drop(skipfeatures,axis=1,inplace=True)


df_test = pd.DataFrame(np.hstack((X_test,y_test[:,None])))
newcols = ["x"+str(i+1) for i in df_test.columns.values]
df_test.columns = newcols
response = newcols[-1]
df_test.rename(columns={response:"y"},inplace=True)
df_test.drop(skipfeatures,axis=1,inplace=True)

###  Forward Stepwise Logistic Regression

- Runs logistic regression using all 2-parameter combinations (within collin_criteria)
- Populate collin_criteria

In [ ]:
collin_criteria = 0.5   # the maximum allowed R2 between terms in a given model

# Runs forward stepwise model search
results,models,scores,sortedmodels,candidates = fsc.ForwardStep_py(df_train,'y',
                    collin_criteria=collin_criteria)

# Identifies model with highest Accuracy
model_sel = results.loc[0,"Model"]
selected_feats = [X_labels.index(i) for i in models[model_sel].terms]
print("\n\nModel with the highest Accuracy:")
print(models[model_sel].formula)
print("1 + "+" + ".join([X_names[X_labels.index(i)] for i in models[candidates[0]].terms])+"\n")

# Finds features used in best model for the training/test data, reruns lr, get statistics
X_train_sel = X_train[:,selected_feats]
X_test_sel = X_test[:,selected_feats]
lr = LogisticRegression()
lr.fit(X_train_sel,y_train)
kfold_score, kfold_stdev = kfold_logreg(X_train_sel, y_train,k=5)
precision, recall, f1 = precision_recall_f1_score(X_train_sel, y_train)
train_R2, test_R2 = calc_mcfadden_R2(X_train_sel, y_train, X_test_sel, y_test)

# Plots model and prints stats
print("Parameters and coefficients:\n{:10.4f} + \n".format(lr.intercept_[0]) + "\n".join(["{:10.4f} * {}".format(lr.coef_[0][i],X_labelname[selected_feats[i]]) for i in range(len(selected_feats))]))
print(f"\nMcFadden Training R2  = {train_R2 :.2f}")
if len(VS) > 0:
    print(f"McFadden Test R2  = {test_R2 :.2f}")
print(f"\nTraining Accuracy   = {100 * lr.score(X_train_sel, y_train):.3f} %")
if len(VS) > 0:  # This is False if no data was left out as a test set. 
    print(f"Test Accuracy  = {100 * lr.score(X_test_sel, y_test):.1f}%")
print("\nTraining K-fold Accuracy = {:.2f} (+/- {:.2f}) %".format(100* kfold_score, 100* kfold_stdev ** 2))
print(f"f1 Score  = {f1:.3f}")
print(f"Precision Score  = {precision :.3f}")
print(f"Recall Score  = {recall:.3f}")

if len(model_sel) > 1:
    heatmap_logreg(selected_feats[0],selected_feats[1],df_train,df_test,results.iloc[0][0],scaler)
else:
    print("The model with the highest accuracy is a 1-parameter model!")

### Display multiple models

- prints out the top n models in a dataframe
- populate within the results1.head() parenthesis

In [ ]:
results['Param_1'] = results.Model.apply(lambda x: X_labelname_dict[x[0]])
results['Param_2'] = results.Model.apply(lambda x: X_labelname_dict[x[1]] if len(x) == 2 else "1-term")
results1 = results.sort_values('McFadden_R2', ascending=False)
results1 = results1.round(2)
results1 = results1[['Model', 'Accuracy', 'McFadden_R2', 'Param_1', 'Param_2', 'n_terms']]

results1.head(10) # put the number of models you want displayed in the parenthesis


In [ ]:
# This cell will print all models with a given parameter

interest_param = 'δ_Cavg'
results2 = results1[(results1.Param_1 == interest_param)| (results1.Param_2 == interest_param)]
results2.head(20)

### Visualize a specific model

- Prints plot and statistics for a model of interest
- populate model number with the index of the model that you want to plot
- populate annotate_train and annotate test

(note that this only plots models that are within the results1 dataframe)

In [ ]:
model_number = 3
annotate_test_set = True    # set to True if you want test set points labeled with ligand ID
annotate_training_set = True    # set to True if you want training set points labeled with ligand ID

model_sel = results.loc[model_number,"Model"]

selected_feats = [X_labels.index(i) for i in models[model_sel].terms]
X_train_sel = X_train[:,selected_feats]
X_test_sel = X_test[:,selected_feats]
lr = LogisticRegression().fit(X_train_sel,y_train)
y_pred_train = lr.predict(X_train_sel)
if len(VS) > 0:  
    test_accuracy = test_accuracy_score(X_test_sel, y_test, X_train_sel, y_train)  
kfold_score, kfold_stdev = kfold_logreg(X_train_sel, y_train, k=5)
precision, recall, f1 = precision_recall_f1_score(X_train_sel, y_train)
train_R2, test_R2 = calc_mcfadden_R2(X_train_sel, y_train, X_test_sel, y_test)

print("Parameters and coefficients:\n{:10.4f} + \n".format(lr.intercept_[0]) + "\n".join(["{:10.4f} * {}".format(lr.coef_[0][i],X_labelname[selected_feats[i]]) for i in range(len(selected_feats))]))
print(f"\nMcFadden Training R2  = {train_R2 :.2f}")
if len(VS) > 0:
    print(f"McFadden Test R2  = {test_R2 :.2f}")
print(f"\nAccuracy  = {100 * lr.score(X_train_sel, y_train):.1f} %")
if len(VS) > 0:
    print(f"Test Accuracy  = {100 * lr.score(X_test_sel, y_test):.1f}%")
print("\nTraining K-fold Accuracy = {:.2f} (+/- {:.2f}) %".format(100* kfold_score, 100* kfold_stdev ** 2))
print(f"f1 Score  = {f1:.3f}")
print(f"Precision Score  = {precision :.3f}")
print(f"Recall Score  = {recall:.3f}")
print('\nNote: \n(1) Black and white points denote active and inactive ligands respectively.')
print('(2) Red and blue denote active and inactive chemical space respectively.')
print('(3) Dashed lines denote 25% and 75% probability that a ligand will be active.')
print('(4) Solid black line denotes 50% probability that a ligand will be active.')


heatmap_logreg(selected_feats[0],selected_feats[1],df_train,df_test,results.iloc[model_number][0], 
               annotate_test=annotate_test_set, annotate_train=annotate_training_set)



### Manual selection of features

- Runs 2D logistic regression with user-defined parameters
- populate features_x, annotate_test_set, annotate_training_set

In [ ]:
features_x = ('x13', 'x28') # tuple of features. eg ('x1', 'x10')
annotate_test_set = False    # set to True if you want to test set points labeled with ligand id 
annotate_training_set = False    # set to True if you want to training set points labeled with ligand id 


selected_feats = sorted([X_labels.index(i.strip()) for i in features_x])
X_train_sel = X_train[:,selected_feats]
X_test_sel = X_test[:,selected_feats]
lr = LogisticRegression().fit(X_train_sel,y_train)
y_pred_train = lr.predict(X_train_sel)
if len(VS) > 0:  
    test_accuracy = test_accuracy_score(X_test_sel, y_test, X_train_sel, y_train)  
kfold_score, kfold_stdev = kfold_logreg(X_train_sel, y_train, k=5)
precision, recall, f1 = precision_recall_f1_score(X_train_sel, y_train)
train_R2, test_R2 = calc_mcfadden_R2(X_train_sel, y_train, X_test_sel, y_test)

print("Parameters and coefficients:\n{:10.4f} + \n".format(lr.intercept_[0]) + "\n".join(["{:10.4f} * {}".format(lr.coef_[0][i],X_labelname[selected_feats[i]]) for i in range(len(selected_feats))]))
print(f"\nMcFadden Training R2  = {train_R2 :.2f}")
if len(VS) > 0:
    print(f"McFadden Test R2  = {test_R2 :.2f}")
print(f"\nAccuracy  = {100 * lr.score(X_train_sel, y_train):.1f} %")
if len(VS) > 0:
    print(f"Test Accuracy  = {100 * lr.score(X_test_sel, y_test):.1f}%")
print("\nTraining K-fold Accuracy = {:.2f} (+/- {:.2f}) %".format(100* kfold_score, 100* kfold_stdev ** 2))
print(f"f1 Score  = {f1:.3f}")
print(f"Precision Score  = {precision :.3f}")
print(f"Recall Score  = {recall:.3f}")
print('\nNote: \n(1) Black and white points denote active and inactive ligands respectively.')
print('(2) Red and blue denote active and inactive chemical space respectively.')
print('(3) Dashed lines denote 25% and 75% probability that a ligand will be active.')
print('(4) Solid black line denotes 50% probability that a ligand will be active.')


heatmap_logreg(selected_feats[0],selected_feats[1],df_train,df_test,features_x, annotate_test=annotate_test_set, annotate_train=annotate_training_set)


## 3.1.1 Virtual screening

### Setup

- Re-makes descriptors dataframe and combines training and test sets
- Check for your model; if LOOCV approach, check for most common descriptors and match model number from above

In [ ]:
ci = pd.read_excel(comp_file+'.xlsx', comp_sheet,index_col=0,header=1,engine='openpyxl')
X_all_names = np.array(ci.SMILES)
compinp = ci[ci.columns[1:]].loc[ci.index[:]]
compinp.dropna(axis=0,inplace=True)
X_all = np.array(compinp)
X_all_ids = np.array(compinp.index)

# add all of the results to the training set
print('Number of samples in original training set = ',len(y_train))
y_train = np.concatenate((y_train,y_test), axis=0)
X_train = np.concatenate((X_train,X_test), axis=0)
print('Number of samples in updated training set = ',len(y_train))

X_screen = X_all
X_ids = X_all_ids.astype(str)

In [ ]:
use_model = 0 # index of model you want to use from results1 (int)
model_sel = results.loc[use_model,"Model"] # choose the model

# collect the model features for the training and screening sets 
selected_feats = [X_labels.index(i) for i in models[model_sel].terms]
X_train_sel = X_train[:,selected_feats]
X_screen_sel = X_screen[:,selected_feats]

# perform lr on training set, then use model to predict training set (for R2) and screening set - gives points on the line
lr = LogisticRegression().fit(X_train_sel,y_train)
y_pred_train = lr.predict_proba(X_train_sel)[:,1]
y_pred_screen =  lr.predict_proba(X_screen_sel)[:,1]

# Pull McFadden R2 
train_R2, test_R2 = calc_mcfadden_R2(X_train_sel, y_train, X_test_sel, y_test) # note that test_R2 here is a meaningless number
print(f"\nMcFadden R2  = {train_R2 :.2f}")

### Create table of virtual screening results with the visualized model

In [ ]:
X_combined = []
for i in range(0,len(X_ids)):
    j = str(X_all_names[i]) + " (ID " + X_ids[i] + ")"
    X_combined.append(j)

df_virtual = pd.DataFrame(list(zip(X_combined, y_pred_screen)), columns = ['Alkyne','Predicted_probability']).sort_values('Predicted_probability', ascending=False)
df_prediction = df_virtual.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
df_prediction.set_properties(**{'text-align': 'center'}).hide_index()
df_virtual.to_excel('Predictions.xlsx', index = True)
df_virtual.head(10)

## 3.2. Leave-One-Out CV Approach
- For small dataset (<50 samples), Uses Bootstrapping
- df_combined from single descriptor logistic regression

### Adapted Plotting Function

In [ ]:
def heatmap_logreg_loo(X, y, feat_idx, X_names, colormap_scheme='RdBu', point_size=200):
    """
    X: numpy array with shape (n_samples, n_features)
    y: labels (0/1)
    feat_idx: list of 2 indices of selected features
    X_names: list of feature names (for axes)
    """
    # Extract selected features
    X_pair = X[:, feat_idx]
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_pair)
    
    # Fit logistic regression
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_scaled, y)
    
    intercept = lr.intercept_[0]
    c1, c2 = lr.coef_[0]
    
    # Create grid for heatmap
    xx = np.linspace(X_scaled[:,0].min()-0.5, X_scaled[:,0].max()+0.5, 500)
    yy = np.linspace(X_scaled[:,1].min()-0.5, X_scaled[:,1].max()+0.5, 500)
    XX, YY = np.meshgrid(xx, yy)
    grid = np.c_[XX.ravel(), YY.ravel()]
    probs = lr.predict_proba(grid)[:,1].reshape(XX.shape)
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(7.25,6))
    heatmap = ax.imshow(probs, origin='lower',
                        extent=[xx.min(), xx.max(), yy.min(), yy.max()],
                        cmap=colormap_scheme, alpha=0.5, aspect='auto')
    
    # Decision boundary and 25/75% probability lines
    x_vals = np.linspace(xx.min(), xx.max(), 500)
    y_dec = -(intercept + c1*x_vals)/c2
    y_75 = (np.log(3) - intercept - c1*x_vals)/c2
    y_25 = (np.log(1/3) - intercept - c1*x_vals)/c2
    ax.plot(x_vals, y_dec, color='black')
    ax.plot(x_vals, y_75, color='black', linestyle='--')
    ax.plot(x_vals, y_25, color='black', linestyle='--')
    
    # Overlay points with black and white circles
    for label, facecolor in zip([0,1], ['white', 'black']):
        ax.scatter(X_scaled[y==label,0], X_scaled[y==label,1],
                   facecolors=facecolor, edgecolors='black', 
                   marker='o', s=point_size, label=f"Class {label}")
    
    # Labels and style
    ax.set_xlabel(X_names[feat_idx[0]], fontsize=18)
    ax.set_ylabel(X_names[feat_idx[1]], fontsize=18)
    ax.tick_params(length=12, width=3)
    for spine in ['top','bottom','left','right']:
        ax.spines[spine].set_linewidth(3)
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax.yaxis.set_major_locator(plt.MaxNLocator(4))
    
    cbar = plt.colorbar(heatmap, ax=ax)
    cbar.set_label('Probability', size=18)
    cbar.ax.tick_params(labelsize=15)
    
    plt.tight_layout()
    fig.savefig('Logistic_Regression_LOO.tif', dpi=600)
    plt.show()

### Determine best two feature regressions for every single fold in an LOO loop

In [ ]:
# ============================================================
# 0. Imports
# ============================================================

import seaborn as sns
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, matthews_corrcoef
from collections import Counter

# ============================================================
# 1. Helper: Bootstrap Confidence Interval
# ============================================================
def bootstrap_CI(y_true, y_pred, metric_fn, n_bootstrap=1000, random_state=42):
    rng = np.random.default_rng(random_state)
    stats = []
    n = len(y_true)
    for _ in range(n_bootstrap):
        idx = rng.choice(n, n, replace=True)
        stats.append(metric_fn(y_true[idx], y_pred[idx]))
    return np.percentile(stats, [2.5, 97.5])

# ============================================================
# 2. Nested LOOCV using ForwardStep_py for 1-2 parameter models
# ============================================================
def nested_LOOCV_ForwardStep_with_CI(X, y, feature_names, verbose=False):
    loo_outer = LeaveOneOut()
    y_true, y_pred = [], []
    selected_features_per_fold = []

    X_df = pd.DataFrame(X, columns=feature_names)
    y_series = pd.Series(y)

    for i, (train_idx, test_idx) in enumerate(loo_outer.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        if verbose:
            print(f"\nOuter Fold {i+1}/{len(y)}")

        # ===============================
        # Standardize features based on train
        # ===============================
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        X_train_df = pd.DataFrame(X_train_scaled, columns=feature_names)
        X_test_df = pd.DataFrame(X_test_scaled, columns=feature_names)
        y_train_series = pd.Series(y_train)


        # Forward stepwise selection using the first script
        results_fs, models_fs, _, sortedmodels, candidates = fsc.ForwardStep_py(
            data=pd.concat([X_train_df, y_train_series.rename("y")], axis=1),
            response="y",
            reg=LogisticRegression(max_iter=1000),
            collin_criteria=0.5
        )

        # Select the top model (highest accuracy)
        top_model_terms = results_fs["Model"].iloc[0]
        selected_features_per_fold.append([feature_names.index(f) for f in top_model_terms])

        # Fit logistic regression on selected features
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_train[:, [feature_names.index(f) for f in top_model_terms]], y_train)
        y_hat = lr.predict(X_test[:, [feature_names.index(f) for f in top_model_terms]])[0]

        y_true.append(y_test[0])
        y_pred.append(y_hat)

        if verbose:
            print(f"Selected: {top_model_terms}, Accuracy={results_fs['Accuracy'].iloc[0]:.2f}")

    y_true, y_pred = np.array(y_true), np.array(y_pred)

    # Compute metrics
    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred),
        "MCC": matthews_corrcoef(y_true, y_pred)
    }

    # Compute 95% CIs using bootstrap
    CIs = {
        "Accuracy": bootstrap_CI(y_true, y_pred, accuracy_score),
        "Balanced Accuracy": bootstrap_CI(y_true, y_pred, balanced_accuracy_score),
        "F1": bootstrap_CI(y_true, y_pred, f1_score),
        "MCC": bootstrap_CI(y_true, y_pred, matthews_corrcoef)
    }

    return {
        "y_true": y_true,
        "y_pred": y_pred,
        "selected_features_per_fold": selected_features_per_fold,
        "metrics": metrics,
        "CIs": CIs
    }

# ============================================================
# 3. Run Nested LOOCV with Bootstrap CIs
# ============================================================
results = nested_LOOCV_ForwardStep_with_CI(X, y_class, X_labels, verbose=True)

# ============================================================
# 4. Print Metrics with 95% Confidence Intervals
# ============================================================
print("\n==============================")
print(" Final Nested LOOCV Performance with 95% CIs")
print("==============================")
for m, v in results["metrics"].items():
    ci_low, ci_high = results["CIs"][m]
    print(f"{m:20s}: {100*v:6.2f}% (95% CI: {100*ci_low:5.2f}-{100*ci_high:5.2f}%)")

# ============================================================
# 5. Feature Selection Frequencies
# ============================================================
feat_counts = Counter(sum(results["selected_features_per_fold"], []))
freq_df = pd.DataFrame({
    "Feature": [X_labels[i] for i in feat_counts.keys()],
    "Count": list(feat_counts.values())
}).sort_values("Count", ascending=False)
freq_df["Frequency (%)"] = 100 * freq_df["Count"] / len(results["selected_features_per_fold"])
print("\n==============================")
print(" Feature Selection Frequencies")
print("==============================")
print(freq_df)

# ============================================================
# 6. Co-selection Heatmap
# ============================================================
n_features = len(X_labels)
co_matrix = np.zeros((n_features, n_features))
for fold_feats in results["selected_features_per_fold"]:
    for i in range(len(fold_feats)):
        for j in range(len(fold_feats)):
            co_matrix[fold_feats[i], fold_feats[j]] += 1
co_matrix /= len(results["selected_features_per_fold"])

# Generate a mask for the upper triangle
mask = np.zeros_like(co_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(20, 15))
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(3)
ax.tick_params(length=12,width=3)
ax.tick_params(which='minor', length=6, width=3)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(co_matrix, cmap="coolwarm", mask=mask, xticklabels=X_names, yticklabels=X_names, linewidths=.5)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
cbar.ax.tick_params(labelsize=15)

fig.suptitle('Feature Co-Selection Frequency (1–2 Feature Models, LOOCV)', fontsize=25)
plt.tight_layout()
fig.savefig('Co-Selection.tif', dpi=600)
plt.show()

# ============================================================
# 7. Decision Surface for Most Frequent Feature Pair
# ============================================================
pair_counts = Counter(tuple(sorted(f)) for f in results["selected_features_per_fold"] if len(f) == 2)
if len(pair_counts) > 0:
    top_pair = pair_counts.most_common(1)[0][0]
    feat1, feat2 = top_pair
    print(f"\nMost frequently selected pair: {X_labels[feat1]} & {X_labels[feat2]}")

    

### Analyze model performance for any two features and plot

In [ ]:

from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, matthews_corrcoef
import numpy as np

# ===============================
# Define features
# ===============================
feat_names = ['x13', 'x28']
feat_idx = [X_labels.index(f) for f in feat_names]

X_selected = X[:, feat_idx]

# ===============================
# LOOCV
# ===============================
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_idx, test_idx in loo.split(X_selected):
    X_train, X_test = X_selected[train_idx], X_selected[test_idx]
    y_train, y_test = y_class[train_idx], y_class[test_idx]

    # Standard scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Fit logistic regression
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train_scaled, y_train)

    # Predict
    y_hat = lr.predict(X_test_scaled)[0]
    y_true.append(y_test[0])
    y_pred.append(y_hat)

y_true, y_pred = np.array(y_true), np.array(y_pred)

# ===============================
# Metrics
# ===============================
metrics = {
    "Accuracy": accuracy_score(y_true, y_pred),
    "Balanced Accuracy": balanced_accuracy_score(y_true, y_pred),
    "F1": f1_score(y_true, y_pred),
    "MCC": matthews_corrcoef(y_true, y_pred)
}

CIs = {m: bootstrap_CI(y_true, y_pred, fn) for m, fn in 
       zip(metrics.keys(), [accuracy_score, balanced_accuracy_score, f1_score, matthews_corrcoef])}
# ===============================
# Print results
# ===============================
print("\n==============================")
print(f"LOOCV Performance for {X_names[feat_idx[0]]} and {X_names[feat_idx[1]]} with 95% CI")
print("==============================")
for m, v in metrics.items():
    ci_low, ci_high = CIs[m]
    print(f"{m:20s}: {100*v:6.2f}% (95% CI: {100*ci_low:5.2f}-{100*ci_high:5.2f}%)")

heatmap_logreg_loo(X, y_class, feat_idx, X_names)